In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw/EW-time table for Upward.csv')

In [3]:
df.columns 

Index(['Stations ', 'Stations -code', 'Train times', '10102', '10202', '10302',
       '10402', '10502', '10602', '10702', '10802', '10104', '10204', '10304',
       '10404', '10504', '10604', '10704', '10804', '10106', '10206', '10306',
       '10406', '10506', '10606', '10706', '10806', '10108', '10208', '10308',
       '10408', '10508', '10608', '10708', '10808', '10110', '10210', '10310',
       '10410', '10510', '10610', '10710', '10810', '10112', '10212', '10312',
       '10412', '10512', '10612', '10712', '10812', '10114', '10214', '10314',
       '10414', '10514', '10614', '10714', '10814', '10116', '10216', '10316',
       '10516', '10616', '10716', '10118', '10218', '10318', '10518'],
      dtype='object')

In [4]:
df.head(3)

,Stations,Stations -code,Train times,10102,10202,10302,10402,10502,10602,10702,...,10116,10216,10316,10516,10616,10716,10118,10218,10318,10518
0,Torhailoch T12202,EW22,Returning,5:55:39,6:09:39,6:20:39,6:32:11,6:46:46,7:01:46,7:16:46,...,19:46:46,20:01:46,20:16:46,20:31:46,20:46:46,21:01:46,21:16:46,21:31:46,21:46:46,22:01:46
1,Torhailoch T12201,EW22,Returning,6:55:39,7:09:39,7:20:39,7:32:11,7:09:39,7:20:39,7:32:11,...,19:46:41,20:01:41,20:21:41,20:41:41,21:01:41,21:21:41,21:41:41,22:01:41,22:21:41,22:44:41
2,Cocacola,EW21,Departure,7:55:39,8:09:39,8:20:39,8:32:11,6:44:59,6:59:59,7:14:59,...,19:44:59,19:59:59,20:19:59,20:39:59,20:59:59,21:19:59,21:39:59,21:59:59,22:19:59,22:42:59


In [5]:
threshold = 0.9  # 50%
nan_cols = df.columns[df.isna().mean() > threshold]
print(nan_cols)
#drop nan_cols
df = df.drop(columns=nan_cols)


Index([], dtype='object')


In [6]:
# raname columns for better readability
df = df.rename(columns={'Stations -code': 'station_id'})
df = df.rename(columns={'Train times': 'arrival_departure'})
df = df.rename(columns={'Stations ': 'Stations'})



In [12]:
import pandas as pd

# Load data

# Step 1: Add Arrival/Departure column based on row order
df['arr_dep'] = ['Arrival' if i % 2 == 0 else 'Departure' for i in range(len(df))]

# Step 2: Melt train columns
id_vars = ['Stations', 'station_id', 'arrival_departure']
value_vars = [col for col in df.columns if col not in id_vars]
df_melted = df.melt(id_vars=id_vars, value_vars=value_vars,
                    var_name='train_id', value_name='time')

# Step 3: Pivot so we have Arrival & Departure per station/train
df_pivot = df_melted.pivot_table(
    index=['Stations', 'station_id', 'train_id'],
    columns='arrival_departure',
    values='time',
    aggfunc='first'
).reset_index()

# Optional: parse time column to datetime
df_pivot['Arrive'] = pd.to_datetime(df_pivot['Arrive'], errors='coerce')
df_pivot['Departure '] = pd.to_datetime(df_pivot['Departure'], errors='coerce')
df_pivot= df_pivot.rename(columns={'Departure ': 'Departure'})

# Done
print(df_pivot.head())


arrival_departure     Stations station_id train_id              Arrive  \
0                  Ayat T10136        EW1    10102 2025-07-24 05:17:28   
1                  Ayat T10136        EW1    10104 2025-07-24 07:00:36   
2                  Ayat T10136        EW1    10106 2025-07-24 09:00:36   
3                  Ayat T10136        EW1    10108 2025-07-24 11:00:36   
4                  Ayat T10136        EW1    10110 2025-07-24 13:00:36   

arrival_departure Departure Outbound Returning            Departure  
0                   5:20:28      NaN        NaN 2025-07-24 05:20:28  
1                   7:02:00      NaN        NaN 2025-07-24 07:02:00  
2                   9:02:00      NaN        NaN 2025-07-24 09:02:00  
3                  11:02:00      NaN        NaN 2025-07-24 11:02:00  
4                  13:02:00      NaN        NaN 2025-07-24 13:02:00  


C:\Users\nebiy\AppData\Local\Temp\ipykernel_16124\4248649633.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pivot['Arrive'] = pd.to_datetime(df_pivot['Arrive'], errors='coerce')
C:\Users\nebiy\AppData\Local\Temp\ipykernel_16124\4248649633.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pivot['Departure '] = pd.to_datetime(df_pivot['Departure'], errors='coerce')


In [13]:
df_pivot.columns

Index(['Stations', 'station_id', 'train_id', 'Arrive', 'Departure', 'Outbound',
       'Returning ', 'Departure'],
      dtype='object', name='arrival_departure')

In [15]:
df_pivot['Dwell_Time'] = df_pivot['Departure'] - df_pivot['Arrive']

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
df_pivot.sort_values(by=['Dwell_Time', 'Arrival'], inplace=True)


In [ ]:
df_pivot.head(30)

arrival_departure,Stations,station_id,train_id,Arrival,Departure,Returning,Dwell_Time
1285,Sebategna,NS23,20101,2025-07-24 06:08:09,2025-07-24 06:08:34,NaN,0 days 00:00:25
0,Abnet,NS22,20101,2025-07-24 06:10:11,2025-07-24 06:10:36,NaN,0 days 00:00:25
340,Darmar T22010,NS21,20101,2025-07-24 06:12:52,2025-07-24 06:13:17,NaN,0 days 00:00:25
1293,Sebategna,NS23,20201,2025-07-24 06:23:09,2025-07-24 06:23:34,NaN,0 days 00:00:25
8,Abnet,NS22,20201,2025-07-24 06:25:11,2025-07-24 06:25:36,NaN,0 days 00:00:25
877,Meshoulekya,NS15,20101,2025-07-24 06:27:13,2025-07-24 06:27:38,NaN,0 days 00:00:25
348,Darmar T22010,NS21,20201,2025-07-24 06:27:52,2025-07-24 06:28:17,NaN,0 days 00:00:25
1149,Riche,NS14,20101,2025-07-24 06:28:52,2025-07-24 06:29:17,NaN,0 days 00:00:25
1557,Temenja Yazh,NS13,20101,2025-07-24 06:30:42,2025-07-24 06:31:07,NaN,0 days 00:00:25
605,Lancha,NS12,20101,2025-07-24 06:32:38,2025-07-24 06:33:03,NaN,0 days 00:00:25


In [ ]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1625 entries, 1285 to 1624
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   Stations    1625 non-null   object         
 1   station_id  1625 non-null   object         
 2   train_id    1625 non-null   object         
 3   Arrival     1474 non-null   datetime64[ns] 
 4   Departure   1474 non-null   datetime64[ns] 
 5   Returning   129 non-null    object         
 6   Dwell_Time  1474 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), object(4), timedelta64[ns](1)
memory usage: 101.6+ KB


In [ ]:
counts = df_pivot.groupby('Stations').size()
print(counts[counts <= 1])  # stations with 1 or 0 records


Series([], dtype: int64)


In [ ]:
#save df_privot as csv
df_pivot.to_csv('../data/processed/NS_time_table_processed.csv', index=False)

In [ ]:
import pandas as pd
from datetime import datetime

# Assuming your data is loaded as `df` with columns: Stations, Arrival, Departure, etc.
# Example structure:
# df = pd.read_csv("your_data.csv")


# Sort by Stations and Arrival Time
df_sorted = df_pivot.sort_values(['Stations', 'Arrival'])

# Calculate Inter-Arrival Gaps (time between consecutive trains at the same station)
df_sorted['Next_Arrival'] = df_sorted.groupby('Stations')['Arrival'].shift(-1)
df_sorted['Arrival_Gap'] = (df_sorted['Next_Arrival'] - df_sorted['Arrival']).dt.total_seconds() / 60  # in minutes

# Drop NaN (last train at each station has no "next arrival")
df_gaps = df_sorted.dropna(subset=['Arrival_Gap'])

# Aggregate by Stations
station_stats = df_gaps.groupby('Stations')['Arrival_Gap'].agg(
    Avg_Gap='mean',
    Min_Gap='min',  # Busiest station (shortest gap)
    Max_Gap='max',  # Least busy station (longest gap)
    Gap_Std='std'   # Consistency of gaps
).reset_index()

# Sort by Avg_Gap to see busiest stations first
station_stats = station_stats.sort_values('Avg_Gap')

print(station_stats)

                    Stations    Avg_Gap  Min_Gap  Max_Gap   Gap_Std
0                      Abnet  14.560606      6.0     21.0  3.059054
1                        Abo  14.560606      6.0     21.0  3.059054
2                 Adey Ababa  14.560606      6.0     21.0  3.059054
3             Atikilt Tera    14.560606      6.0     21.0  3.059054
4               Autobus Tera  14.560606      6.0     21.0  3.059054
5              Darmar T22010  14.560606      6.0     21.0  3.059054
6              Gojam Berenda  14.560606      6.0     21.0  3.059054
7              Kality T20619  14.560606      6.0     21.0  3.059054
8                     Lancha  14.560606      6.0     21.0  3.059054
9                     Leghar  14.560606      6.0     21.0  3.059054
10  Menilik II Square T22705  14.560606      6.0     21.0  3.059054
11               Meshoulekya  14.560606      6.0     21.0  3.059054
12                    Mexico  14.560606      6.0     21.0  3.059054
13              Nefas Silk 1  14.560606      6.0